In [2]:
import pandas as pd

In [19]:
chalearn_path = "/home/vlados/datasets/chalearn/"
chalearn_labels_path = chalearn_path + "Info_devel_valid.txt"

In [4]:
chalearn_labels_df = pd.read_csv(chalearn_labels_path, sep="\t")

In [5]:
chalearn_labels_df.head()

,Set,Num,Lexicon,UserID,Date,MinDepth,MaxDepth,DepthRes,DepthAcc,Missing
0,devel,1,CanadaAviationGroundCirculation1,A,2011 10 05 16 26,801,1964,76,2,0
1,devel,2,RefereeWrestlingSignals1,I,2011 09 28 14 31,801,1968,71,1,0
2,devel,3,GangHandSignals1,L,2011 09 27 12 16,801,1506,95,1,0
3,devel,4,DivingSignals2,J,2011 09 26 09 01,801,1869,100,1,0
4,devel,5,GestunoDisaster,L,2011 10 11 15 16,824,1964,91,2,0


In [17]:
chlearn_diving_labels_df = chalearn_labels_df.loc[chalearn_labels_df["Lexicon"].isin(["DivingSignals1", "DivingSignals2"])].drop(chalearn_labels_df.loc[:,"UserID":], axis = 1)
chlearn_diving_labels_df

,Set,Num,Lexicon
3,devel,4,DivingSignals2
19,devel,20,DivingSignals1
22,valid,3,DivingSignals2
45,devel,26,DivingSignals2
54,devel,35,DivingSignals1
62,devel,43,DivingSignals2
71,devel,52,DivingSignals1
90,devel,71,DivingSignals1
92,devel,73,DivingSignals1
98,devel,79,DivingSignals2


In [18]:
import os

In [95]:
def find_dir(number, path):
    for dirname in os.listdir(path):
        splitted = dirname.split("-")
        if splitted[0] != "devel":
            continue
        if (int(splitted[1]) < number <= int(splitted[2])):
            subpath = os.path.join(path, dirname)
            for subdirname in os.listdir(subpath):
                subsplitted = subdirname.split("devel")
                if subsplitted[0] != "":
                    continue
                if int(subsplitted[1]) == number:
                    dest_path = os.path.join(subpath, subdirname)
                    for dest_file in os.listdir(dest_path):
                        if dest_file.split(".")[1] == "csv":
                            yield os.path.join(dest_path, dest_file)
                            

In [104]:
df = pd.DataFrame(columns=["video", "label"])
for df_path in find_dir(chlearn_diving_labels_df.iloc[0]["Num"], chalearn_path):
    df = pd.concat([df, pd.read_csv(df_path,names=["video", "label"])], ignore_index=True, sort=False)
df

,video,label
0,devel04_11,3
1,devel04_12,1
2,devel04_13,2 7
3,devel04_14,2
4,devel04_15,9 5 4
5,devel04_16,7
6,devel04_17,4 7 4 3
7,devel04_18,5
8,devel04_19,9 2
9,devel04_20,9 10 8 4


In [52]:

for index, row in chlearn_diving_labels_df.iterrows():
    print(find_dir(row["Num"]))

devel-1-20
devel-1-20
devel-1-20
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-21-135
devel-136-250
devel-136-250
devel-136-250
devel-136-250
devel-136-250
devel-251-365
devel-251-365
devel-251-365
devel-251-365
devel-251-365
devel-251-365
devel-251-365
devel-366-480
devel-366-480
devel-366-480
devel-366-480
